In [4]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense

# EDA 데이터 경로 설정
eda_folder = "EDA_preprocessed/Output_EDA/"

# train, test split
train_ids = ['001', '005', '006', '009', '010', '011', '012', '013', '015', '021', '022', '023',
             '024', '026', '027', '028', '031', '032', '033', '036', '038', '041', '042', '044',
             '045', '047', '053', '054', '055', '060', '062', '063', '064', '066', '070', '072',
             '073', '075', '076', '077', '078']
test_ids = ['002', '003', '007', '017', '018', '019', '020', '025', '029', '030', '039', '043',
            '048', '052', '059', '065', '068', '074']

# 데이터프레임 초기화
X_train = pd.DataFrame()
X_test = pd.DataFrame()
y_train = []
y_test = []

# 데이터 전처리
data_train = []
data_test = []

# 전체 데이터셋에 대한 표준화
scaler = StandardScaler()

for id in train_ids:
    for emotion in ['happy', 'fear', 'sad', 'neutral', 'anger', 'disgust']:
        file_path = f"{eda_folder}/{id}/{id}_{emotion}.csv"
        data = pd.read_csv(file_path, header=None)
        scaled_data = scaler.fit_transform(data)
        data_train.append(scaled_data)
        y_train.append(emotion)  

for id in test_ids:
    for emotion in ['happy', 'fear', 'sad', 'neutral', 'anger', 'disgust']:
        file_path = f"{eda_folder}/{id}/{id}_{emotion}.csv"
        data = pd.read_csv(file_path, header=None)
        scaled_data = scaler.transform(data)
        data_test.append(scaled_data)
        y_test.append(emotion)  



In [12]:
# 데이터를 각각의 리스트에 추가
for i in range(len(train_ids)):
    for j in range(6):
        features = np.concatenate((np.array([train_ids[i]]), data_train[i][j].flatten(), np.array([j])), axis=None)
        X_train = X_train.append(pd.Series(features), ignore_index=True)

for i in range(len(test_ids)):
    for j in range(6):
        features = np.concatenate((np.array([test_ids[i]]), data_test[i][j].flatten(), np.array([j])), axis=None)
        X_test = X_test.append(pd.Series(features), ignore_index=True)


In [13]:
print(X_train.dtypes)
print(X_test.dtypes)

0    object
1    object
2    object
dtype: object
0    object
1    object
2    object
dtype: object


In [14]:
# 데이터 유형 변환
X_train = X_train.astype(float)
X_test = X_test.astype(float)

In [15]:
# 라벨 데이터 변환
label_mapping = {'happy': 0, 'fear': 1, 'sad': 2, 'neutral': 3, 'anger': 4, 'disgust': 5}
y_train = [label_mapping[y] for y in y_train]
y_test = [label_mapping[y] for y in y_test]

In [16]:
print(X_train.dtypes)
print(X_test.dtypes)

0    float64
1    float64
2    float64
dtype: object
0    float64
1    float64
2    float64
dtype: object


In [61]:
# Autoencoder 모델 정의
input_dim = X_train.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim*2, activation='relu')(input_layer)
encoder2 = Dense(encoding_dim, activation='relu')(encoder)
decoder = Dense(encoding_dim*2, activation='relu')(encoder2)
decoder2 = Dense(input_dim, activation='sigmoid')(decoder)

autoencoder = Model(input_layer, decoder2)
autoencoder.compile(optimizer='adam', loss='mse')

# 모델 학습
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

# 특징 추출
encoder_model = Model(input_layer, encoder2)
encoded_X_train = encoder_model.predict(X_train)
encoded_X_test = encoder_model.predict(X_test)


Epoch 1/10
8/8 [==============================] - 1s 57ms/step - loss: 692.4851 - val_loss: 5350.8989
Epoch 2/10
8/8 [==============================] - 0s 28ms/step - loss: 691.8301 - val_loss: 5350.0718
Epoch 3/10
8/8 [==============================] - 0s 24ms/step - loss: 691.7368 - val_loss: 5349.5581
Epoch 4/10
8/8 [==============================] - 0s 10ms/step - loss: 691.6962 - val_loss: 5348.6631
Epoch 5/10
8/8 [==============================] - 0s 12ms/step - loss: 691.6662 - val_loss: 5347.5581
Epoch 6/10
8/8 [==============================] - 0s 12ms/step - loss: 691.5668 - val_loss: 5346.7939
Epoch 7/10
8/8 [==============================] - 0s 10ms/step - loss: 691.5410 - val_loss: 5346.7217
Epoch 8/10
8/8 [==============================] - 0s 17ms/step - loss: 691.4851 - val_loss: 5346.7871
Epoch 9/10
8/8 [==============================] - 0s 32ms/step - loss: 691.4518 - val_loss: 5346.6963
Epoch 10/10
4/4 [==============================] - 0s 5ms/step


In [50]:
# 결과 확인
print("Encoded Train data:")
print(encoded_X_train)
print("Encoded Test data:")
print(encoded_X_test)

Encoded Train data:
[[6.31565303e-02 1.56413853e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.44076630e-01]
 [1.04778215e-01 4.03436750e-01 0.00000000e+00 ... 3.86607181e-03
  0.00000000e+00 1.78315297e-01]
 [0.00000000e+00 1.32615292e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 4.91463929e-01]
 ...
 [5.93901730e+00 6.14910030e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 9.11300564e+00]
 [5.82573175e+00 6.40259600e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 9.22798252e+00]
 [5.78646994e+00 6.56912184e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 9.30910683e+00]]
Encoded Test data:
[[ 0.         0.         0.        ...  0.         0.         1.6483961]
 [ 0.         0.         0.        ...  0.         0.         1.7154598]
 [ 0.         0.         0.        ...  0.         0.         1.772403 ]
 ...
 [ 0.        91.08705    0.        ...  0.         0.        60.42357  ]
 [ 0.        91.61652    0.        ...  0.         0.        60.